***Fundamentals of Artificial Intelligence***

> **Lab 6:** *Natural Language Processing and Chat Bots* <br>

> **Performed by:** *Corneliu Catlabuga*, group *FAF-213* <br>

> **Verified by:** Elena Graur, asist. univ.

#### Imports

In [ ]:
import os
import pandas as pd
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import AutoTokenizer, T5ForConditionalGeneration
from tqdm import tqdm
from warnings import filterwarnings

filterwarnings("ignore")

#### Task 1

Set up the Telegram Bot. Interact with BotFather on Telegram to obtain an API token. Create your Telegram Bot (its name should follow the pattern FIA_Surname_Name_FAF_21x). Make sure you are able to receive and send requests to it.

1. Bot link: [FIA_Catlabuga_Corneliu_FAF_213](https://t.me/FAFCatlabugaCorneliuFAF213bot)

2. Run `app.py` to start the bot.

#### Task 2

Create a dataset that will serve as a training set for your model. It should follow the rules:
- an entry consists of two parts: the question and the answer;
- there are at least 75 entries written by you in your dataset;
- questions should be something tourists or locals can ask about a new city.

You can increase your dataset by adding open-source data. However, you MUST clearly show the questions written by you. Split your dataset into train and validation.

*Hint: it is recommended to split it into 80% and 20%, but you can adjust it according to your needs.*

#### Dataset

In [28]:
dataset = pd.read_csv('dataset.csv')

questions = dataset['question'].tolist()
answers = dataset['answer'].tolist()

#### Task 3

Use Tensorflow or Pytorch to implement the architecture of the Neural Network you are planning to use. It is highly recommended to use a Seq2Seq model (implement an LSTM or GRU architecture). You are NOT allowed to use pre-built or existing solutions (yep, connecting to GPT will not work).

In [29]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

def tokenize(data):
    return tokenizer(data, padding=True, truncation=True, return_tensors="pt")

In [30]:
class Seq2SeqDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'labels': self.targets['input_ids'][idx]
        }

In [31]:
tokenized_questions = tokenize(questions)
tokenized_answers = tokenize(answers)

dataset = Seq2SeqDataset(tokenized_questions, tokenized_answers)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

#### Task 4

Train your model and fine-tune it based on the chosen performance metrics.

In [32]:
class Seq2SeqModel(nn.Module):
    def __init__(self, model_name="t5-base"):
        super(Seq2SeqModel, self).__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask, labels):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

In [33]:
def train(epochs: int = 10, file_name: str = 'model.pth'):
    model = Seq2SeqModel()
    optimizer = optim.AdamW(model.parameters(), lr=5e-4)

    for epoch in range(epochs):
        model.train()
        loop = tqdm(dataloader, leave=True)

        for batch in loop:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update progress bar
            loop.set_description(f"Epoch {epoch}")
            loop.set_postfix(loss=loss.item())

    os.makedirs('models', exist_ok=True)
    torch.save(model.state_dict(), f'models/{file_name}')

In [34]:
train(5, 'model10base.pth')

Epoch 4: 100%|██████████| 6/6 [00:37<00:00,  6.23s/it, loss=0.873]


#### Task 5

Integrate your model into your Telegram ChatBot, so that the sent messages are taken as input by the model and its output is sent back as a reply.

In [36]:
model = Seq2SeqModel()
model.load_state_dict(torch.load('models/model10base.pth'))

def generate_answer(question, model, tokenizer):
    model.eval()
    input_ids = tokenizer(question, return_tensors="pt").input_ids
    output = model.model.generate(input_ids)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test example
test_question = "Can I visit the flereek?"
display(generate_answer(test_question, model, tokenizer))

/tmp/ipykernel_42089/2486391683.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/model10base.pth'))


'Yes, but only after completing a short orientation.'

#### Task 6

Handle potential errors that may occur, such as model errors or invalid inputs.